In [1]:
import os; os.chdir("../")
import sys; sys.path.append("..")

In [2]:
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from pnpxai import AutoExplanationForImageClassification

from helpers import get_imagenet_dataset, get_torchvision_model, denormalize_image

# ------------------------------------------------------------------------------#
# -------------------------------- basic usage ---------------------------------#
# ------------------------------------------------------------------------------#

# setup
torch.set_num_threads(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model, transform = get_torchvision_model('resnet18')
dataset = get_imagenet_dataset(transform, indices=range(5))
loader = DataLoader(dataset, batch_size=4, shuffle=False)

# create auto explanation
expr = AutoExplanationForImageClassification(
    model=model.to(device),
    data=loader,
    input_extractor=lambda batch: batch[0].to(device),
    label_extractor=lambda batch: batch[-1].to(device),
    target_extractor=lambda outputs: outputs.argmax(-1).to(device),
    target_labels=False,  # target prediction if False
)


# browse the recommended
expr.recommended.print_tabular()  # recommendation
expr.recommended.explainers  # -> List[Type[Explainer]]

# browse explainers and metrics
expr.manager.explainers  # -> List[Explainer]
expr.manager.metrics  # -> List[Metric]

# -> Explainer. In this case, LRPEpsilonGammaBox
expr.manager.get_explainer_by_id(7)
# -> PostProcessor. In this case, PostProcessor(pooling_method='sumpos', normalization_method='minmax')
expr.manager.get_postprocessor_by_id(0)
expr.manager.get_metric_by_id(0)  # -> Metric. In this case, AbPC

# explain and evaluate
results = expr.run_batch(
    explainer_id=4,
    postprocessor_id=0,
    metric_id=0,
    data_ids=range(4),
)

#------------------------------------------------------------------------------#
#------------------------------- optimization ---------------------------------#
#------------------------------------------------------------------------------#

# user inputs
explainer_id = 4 # explainer_id to be optimized: KernelShap
metric_id = 1 # metric_id to be used as objective: AbPC
data_id = 3


# plots
n_explainers = len(expr.manager.explainers)
edge_size = 3
fig, axes = plt.subplots(n_explainers, 4, figsize=(edge_size*4, edge_size*n_explainers))

for explainer_id in range(n_explainers):
    explainer_nm = expr.manager.get_explainer_by_id(explainer_id).__class__.__name__

    data = expr.manager.batch_data_by_ids(data_ids=[data_id])
    inputs = expr.input_extractor(data)
    labels = expr.label_extractor(data)

    # vis inputs
    r = explainer_id
    axes[r, 0].imshow(
        denormalize_image(
            inputs[0].detach().cpu(),
            mean=transform.mean,
            std=transform.std,
    ))
    axes[r, 0].set_ylabel(explainer_nm)

#     # vis explanations
#     for c, (title, trial) in enumerate(trials_to_vis.items(), 1):
#         explainer = trial.user_attrs['explainer']
#         postprocessor = trial.user_attrs['postprocessor']

#         attrs = explainer.attribute(inputs, labels)
#         pps = postprocessor(attrs)
#         axes[r, c].imshow(pps[0].detach().cpu(), cmap='twilight')
#         if r == 0:
#             axes[r, c].set_title(title)

# fig.subplots_adjust(wspace=-.5, hspace=-.5)
# fig.tight_layout()

# for ax in axes.flatten():
#     ax.set_xticks([])
#     ax.set_yticks([])


2025-05-27 15:15:11.246609: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-27 15:15:15.987342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748326517.767644 1352447 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748326518.121076 1352447 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-27 15:15:19.572214: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

ImportError: cannot import name 'AutoExplanationForImageClassification' from 'pnpxai' (/data9/cwl/repo/pnpxai/research/../pnpxai/__init__.py)